This notebook applies web scraping methodologies to get values and features of Argentina car prices from Mercado Libre.

#1. Import modules

In [ ]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import time
import requests
!pip install requests_html
import requests_html
from requests_html import HTMLSession

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 KB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 5.4 MB/s eta 0:00:00
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=a5dc99008fe1df001eb45e374ec98f571e484cadb28e89dd25a148e70c77d76d
  Stored in directory: /root/.cache/pip/wheels/e5/33/1f/68392720485b3ecf125a69e700baaab7624616deedea2fa6e2
Successfully built parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
string = input('¿Qué quieres buscar?')
r = requests.get('https://listado.mercadolibre.com.ar/{}#D[A:{}]'.format(string.replace(' ', '-'), string))
contenido = r.content

soup = BeautifulSoup(contenido, 'html.parser')

# Última página
try:
    last_page = soup.find('li', {'class': 'andes-pagination__page-count'}).text
    last_page_modified = int(last_page.replace('de ',''))
except:
    pass

# Array para añadir los objetos
products_array = []

for page in range(0, last_page_modified):
    
    initial_result = requests.get('https://listado.mercadolibre.com.ar/{}_Desde_{}_NoIndex_True'.format(string.replace(' ', '-'), (page*50)+1))
    content_pagination = initial_result.content
    
    soup_pagination = BeautifulSoup(content_pagination, 'html.parser')
    
    alldivs = soup_pagination.find_all('div', {'class': 'andes-card'})
    
    for item in alldivs:
        data = {}
        data['nombre articulo'] = item.find('h2', {'class': 'ui-search-item__title'}).text
        data['precio'] = item.find('span', {'class': 'price-tag-amount'}).text
        try:
            data['link'] = item.find('a', {'class': 'ui-search-result__content'})['href']
        except: 
            data['link'] = item.find('a', {'class': 'ui-search-link'})['href']

        products_array.append(data)

print(len(products_array))
print(products_array)

df = pd.DataFrame(products_array)



In [ ]:
df.head()

In [ ]:
#cars
df[['company','model','year','version','color','fuel','doors','transmisions','motor','bodywork','km']]= '.'

#Apartment
#df[['sup_total','sup_cub','ambient','bedrooms','bathroom','disposition','orientation']]= '.'

In [ ]:
df.head(2)

In [ ]:
link_list = df['link']
session = HTMLSession()
url = link_list[0]
r=session.get(url)
names=r.html.find(".andes-table__header")
table=r.html.find("span.andes-table__column--value")
table
for i in range(len(table)):
    print(f"Name : {names[i].text}\nValue : {table[i].text}\n-------------------------------------------------")

In [ ]:
list(df)

In [ ]:
for i in range(0,len(link_list)):
  url = link_list[i]
  r=session.get(url)
  names=r.html.find(".andes-table__header")
  table=r.html.find("span.andes-table__column--value")
  try:
    df.at[i, 'company'] = table[0].text
    df.at[i, 'model'] = table[1].text
    df.at[i, 'year'] = table[2].text
    df.at[i, 'version'] = table[3].text
    df.at[i, 'color'] = table[4].text
    df.at[i, 'fuel'] = table[5].text
    df.at[i, 'doors'] = table[6].text
    df.at[i, 'transmisions'] = table[7].text
    df.at[i, 'motor'] = table[8].text
    df.at[i, 'bodywork'] = table[9].text
    df.at[i, 'km'] = table[10].text
  except:
    continue
  print(i)
  


In [ ]:
df.head()

In [ ]:
# Guardar los resultados obtenidos en csv
df.to_csv("data_{}.csv".format(string))